In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
characters = pd.read_csv('marvel_characters.csv')
titles = pd.read_csv('marvel_titles.csv')

In [3]:
G = nx.Graph()

In [4]:
for index, row in titles.iterrows():
  G.add_node(row['title'] + '*', bipartite=0, media_type=row['type'], release_year=row['release_year'], release_month=row['release_month'], release_day=row['release_day'], rating=row['rating'], length=row['length'], starts=row['stars'], genres=row['genres'])

In [5]:
for index, row in characters.iterrows():
  if row['title'] == 'What If...?':
    char_name = row['character'] + '-AR'
  else:
    char_name = row['character']
  G.add_node(char_name, bipartite=1, alias=row['character_alias'], actor=row['actor'])

In [6]:
for index, row in characters.iterrows():
  if row['title'] == 'What If...?':
    char_name = row['character'] + '-AR'
  else:
    char_name = row['character']
  G.add_edge(row['title'] + '*', char_name)

In [8]:
character_nodes = {n for n, d in G.nodes(data=True) if d['bipartite'] == 1}
title_nodes = set(G) - character_nodes

In [9]:
nx.is_bipartite(G)

True

In [10]:
G_char = nx.bipartite.projected_graph(G, character_nodes)
print(G_char)

Graph with 794 nodes and 14793 edges


In [11]:
c = int(np.round(sum([d for (n,d) in nx.degree(G_char)]) / G_char.number_of_nodes()))
clustering = nx.average_clustering(G_char)

In [12]:
small_world = nx.Graph()
for i in range(G_char.number_of_nodes()):
  small_world.add_node(i)
  if small_world.number_of_nodes() > 1:
    small_world.add_edge(list(small_world.nodes)[-1], list(small_world.nodes)[-2])
small_world.add_edge(list(small_world.nodes)[-1], list(small_world.nodes)[0])

if c % 2 == 0:
  k = c
else:
  k = c - 1


In [13]:
print(small_world)

Graph with 794 nodes and 794 edges


In [14]:
# Create lattice
size = int(k/2)
num_nodes = small_world.number_of_nodes()
for node in list(small_world.nodes()):
  for i in range(1, size + 1):
    if small_world.has_edge(node, node - i) is False and node - i > 0:
      small_world.add_edge(node, node - i)
  for i in range(1, size + 1):
    if node + i < num_nodes:
      if small_world.has_edge(node, node + i) is False:
        small_world.add_edge(node, node + i)
    else:
      target = node + i - num_nodes
      if small_world.has_edge(node, target) is False:
        small_world.add_edge(node, target)

In [15]:
print(small_world)

Graph with 794 nodes and 14292 edges


In [16]:
# Calculate p (beta)
c0 = nx.average_clustering(small_world)
C = nx.average_clustering(G_char)
p = 1 - (C/c0)**(1/3)
print(p)

-0.08457597156109453


In [17]:
# Rewire
from numpy.random import choice
from random import random
node_list = list(small_world.nodes())
for node in list(small_world.nodes()):
  counter = 0
  for edge in small_world.edges(node):
    counter += 1
    if (edge[0] == node and edge[1] > node) or (edge[1] == node and edge[0] > node):
      target = choice(node_list)
      if target != node and small_world.has_edge(node, target) is False:
        if random() < p:
          counter += 1
          small_world.remove_edge(edge[0], edge[1])
          small_world.add_edge(node, target)
  # print(counter)


In [18]:
# Clustering Coefficient
print('Real:', nx.average_clustering(G_char))
print('Simulated:', nx.average_clustering(small_world))

Real: 0.9295057308049338
Simulated: 0.7285714285714371


In [19]:
# Average Degree
print('Real:', sum([d for (n,d) in nx.degree(G_char)]) / G_char.number_of_nodes())
print('Simulated:', sum([d for (n,d) in nx.degree(small_world)]) / small_world.number_of_nodes())

Real: 37.26196473551637
Simulated: 36.0


In [21]:
#Diameter of largest component
components = sorted(nx.connected_components(small_world), key=len, reverse=True)
diameter = nx.diameter(small_world.subgraph(components[0]))
print('Diameter of largest connected component:', diameter)

Diameter of largest connected component: 23


In [22]:
# Average shortest path length of largest component
components = sorted(nx.connected_components(small_world), key=len, reverse=True)
nx.average_shortest_path_length(small_world.subgraph(components[0]))

11.514501891551072